In [25]:
import json
from pathlib import Path

from FbxCommon import InitializeSdkObjects, LoadScene

def load_fbx(file_path):
    # Initialize the SDK manager and scene
    sdk_manager, scene = InitializeSdkObjects()

    # Load the FBX file
    success = LoadScene(sdk_manager, scene, file_path)
    if not success:
        print(f"Failed to load FBX file: {file_path}")
        return None

    print(f"Successfully loaded FBX file: {file_path}")
    return scene

# Example usage
file_path = "BP_Mixamo_New10_Scene_1_16_0.FBX"  # Replace with the actual path to your FBX file
scene = load_fbx(file_path)

Successfully loaded FBX file: BP_Mixamo_New10_Scene_1_16_0.FBX


In [26]:
from fbx import FbxCriteria, FbxAnimStack, FbxAnimLayer

KFCURVENODE_T_X = "X"
KFCURVENODE_T_Y = "Y"
KFCURVENODE_T_Z = "Z"

KFCURVENODE_R_X = "X"
KFCURVENODE_R_Y = "Y"
KFCURVENODE_R_Z = "Z"
KFCURVENODE_R_W = "W"

KFCURVENODE_S_X = "X"
KFCURVENODE_S_Y = "Y"
KFCURVENODE_S_Z = "Z"


def getCurveKeys(pCurve):
    if pCurve:
        keyframes_values = []
        lKeyCount = pCurve.KeyGetCount()

        for lCount in range(lKeyCount):
            lKeyValue = pCurve.KeyGetValue(lCount)
            keyframes_values.append(float(lKeyValue))
    else:
        keyframes_values = None
    return keyframes_values


motion_data = {}

anim_stack = scene.GetSrcObject(FbxCriteria.ObjectType(FbxAnimStack.ClassId), 0)
anim_layer = anim_stack.GetSrcObject(FbxCriteria.ObjectType(FbxAnimLayer.ClassId), 0)

root_node = scene.GetRootNode().GetChild(0)

motion_data = {}
joints_names = []
def append_node_data(node):
    node_name = node.GetName()
    joints_names.append(node_name)

    # Register animation data
    motion_data[node_name + '_TX'] = getCurveKeys(node.LclTranslation.GetCurve(anim_layer, KFCURVENODE_T_X))
    motion_data[node_name + '_TY'] = getCurveKeys(node.LclTranslation.GetCurve(anim_layer, KFCURVENODE_T_Y))
    motion_data[node_name + '_TZ'] = getCurveKeys(node.LclTranslation.GetCurve(anim_layer, KFCURVENODE_T_Z))
    motion_data[node_name + '_RX'] = getCurveKeys(node.LclTranslation.GetCurve(anim_layer, KFCURVENODE_R_X))
    motion_data[node_name + '_RY'] = getCurveKeys(node.LclTranslation.GetCurve(anim_layer, KFCURVENODE_R_Y))
    motion_data[node_name + '_RZ'] = getCurveKeys(node.LclTranslation.GetCurve(anim_layer, KFCURVENODE_R_Z))


   # call recursively for children
    child_count = node.GetChildCount()
    if child_count > 0:
        # call recursively
        for child_idx in range(child_count):
            child_node = node.GetChild(child_idx)
            append_node_data(child_node)
    return motion_data

motion_data = append_node_data(root_node)

In [27]:
def get_skeleton_hierarchy(root_node):
    """
    Returns the hierarchy of the skeleton as a list of indices,
    where each index represents the parent bone index.

    Parameters:
    - root_node: The root node of the skeleton hierarchy.

    Returns:
    - list: An array-like structure representing the parent indices of each bone.
    """
    hierarchy = []
    node_index_map = {}  # Maps node unique IDs to their indices

    def traverse(node, parent_index):
        # Assign an index to the current node
        current_index = len(hierarchy)
        node_id = node.GetUniqueID()  # Use unique ID as the key
        node_index_map[node_id] = current_index

        # Add the parent index to the hierarchy
        hierarchy.append(parent_index)

        # Traverse the children
        for i in range(node.GetChildCount()):
            child_node = node.GetChild(i)
            traverse(child_node, current_index)

    # Start traversal from the root node
    traverse(root_node, -1)

    return hierarchy


In [28]:
data = {'motion_data': motion_data,
        'joints_names': joints_names,
        'parents': get_skeleton_hierarchy(root_node)
        }

with open(Path(file_path).stem + '.json', 'w') as fp:
    json.dump(data, fp, indent=2)